Step 1 : Project Setup

In [7]:
import pandas as pd  # For data manipulation and analysis
import logging  # For proper logging and debugging 
from typing import Dict, Any  # Type hints for better code maintainability
import os
from dotenv import load_dotenv # For loading environment variable files
load_dotenv() # Load environment variables from a .env file

# LangChain components
from langchain_core.tools import tool  # For creating callable tools
from langchain_core.runnables import Runnable  # For creating composable chains
from langchain.chat_models import init_chat_model  # For accessing Groq's LLMs
from langchain_core.prompts import ChatPromptTemplate  # For structured prompting
from langchain_core.output_parsers import PydanticOutputParser  # For structured outputs
from pydantic import BaseModel, Field  # For data validation and typing

file_path = os.getenv("file_location")
if file_path is None:
    raise ValueError("FILE_PATH environment variable not set. Please set it to the path of your CSV file.")


logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s'
)

Step_2 : Data Preparation

In [8]:
# Load the dataset
try:
    df = pd.read_csv(file_path)
    
    # Log basic information about the DataFrame
    logging.info(f"Successfully loaded DataFrame with {len(df)} rows and {len(df.columns)} columns")
    logging.info(f"Columns: {df.columns.tolist()}")
except Exception as e:
    logging.error(f"Error loading data: {e}")
    raise

INFO - Successfully loaded DataFrame with 1000 rows and 17 columns
INFO - Columns: ['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income', 'Rating']


Step_3 : Function to extract and store the schema information 

When working with AI and data, you need to create a bridge between what the data actually contains and what the AI understands about it. This initial exploration helps us build that bridge by identifying what context to provide to the LLM.

In [9]:
def extract_schema_info(dataframe: pd.DataFrame) -> Dict[str, Any]:
    """
    Extract schema information from DataFrame to help the LLM understand the data.
    
    Args:
        dataframe: The pandas DataFrame to analyze
        
    Returns:
        Dictionary containing schema information
    """
    return {
        "columns": dataframe.columns.tolist(),
        "dtypes": {col: str(dtype) for col, dtype in zip(dataframe.columns, dataframe.dtypes)},
        "sample_data": dataframe.head(3).to_dict('records'),
        "summary": {
            "rows": len(dataframe),
            "columns": len(dataframe.columns),
            "numeric_columns": dataframe.select_dtypes(include=['number']).columns.tolist(),
            "categorical_columns": dataframe.select_dtypes(include=['object']).columns.tolist(),
            "datetime_columns": dataframe.select_dtypes(include=['datetime']).columns.tolist()
        }
    }

schema_info = extract_schema_info(df)

DataFrame Query Tool

The @tool decorator marks this function as a "tool" that can be used by a LangChain agent.


This means an AI model can call this function to perform specific tasks (like querying a DataFrame).

In [10]:
@tool
def query_dataframe(code: str) -> str:
    """
Executes pandas code on the DataFrame 'df' and returns the result.

Args:
    code: A string containing pandas code to execute (e.g., df['Price'].mean())

Returns:
    String representation of the query result
"""
    try:
        # Safe execution using eval instead of exec
        # eval directly returns the value, making it better for expressions
        result = eval(code, {"__builtins__": {}}, {"df": df, "pd": pd})
        
        # Format the output based on result type
        if isinstance(result, pd.DataFrame):
            if len(result) > 10:
                return f"DataFrame with {len(result)} rows:\n{result.head(5).to_string()}\n...(showing only first 5 rows)"
            else:
                return result.to_string()
        elif isinstance(result, pd.Series):
            if len(result) > 10:
                return f"Series with {len(result)} elements:\n{result.head(5).to_string()}\n...(showing first 5 elements)"
            else:
                return result.to_string()
        else:
            return str(result)
    except Exception as e:
        logging.error(f"Error executing query: {code}, Error: {e}")
        return f"Error: {e}"

In [11]:
def init_model():
    """
    Initialize the LLM with appropriate settings and tools.
    
    Returns:
        Configured LLM instance with tools bound
    """
    try:
        api_key = os.getenv("GROQ_API_KEY")
        if api_key is None:
            raise ValueError("GROQ_API_KEY environment variable not set. Please set it to your Groq API key.")
        #logging.info("Initializing LLM with Groq API key...")
        model = init_chat_model(
            model="qwen-qwq-32b",
            temperature=0.5,
            max_tokens=2000,
            api_key=api_key,
            model_provider="groq")


        # Bind the DataFrame query tool to the model
        model_with_tools = model.bind_tools([query_dataframe])
        
        #logging.info("LLM initialized successfully with tools")
        
        return model_with_tools
    except Exception as e:
        logging.error(f"Failed to initialize LLM: {e}")
        raise

Creating Effective Prompts

“Prompt engineering” is overhyped. It’s not just good English and using AI tools—it requires actual understanding and basic coding. Without that, you’re just buying overpriced courses to ask ChatGPT how to do the job.

In [12]:
def create_prompt_templates():
    """Create the prompt templates for the agent."""
    
    # Query generation prompt - converts natural language to pandas code
    query_prompt = ChatPromptTemplate.from_template("""
    You are a pandas expert who converts questions about data into executable pandas code.

    DataFrame Information:
    - Columns: {columns}
    - Data types: {dtypes}
    - Sample data: {sample_data}
    
    Important instructions:
    1. ONLY write pandas code that queries the DataFrame named 'df'
    2. Your code should be a single expression that can be executed with eval()
    3. Do NOT include assignments, print statements, or multiple lines of code
    4. For aggregations, handle NaN values appropriately
    5. Make your code efficient and accurate
    
    User question: {question}
    
    Respond by using the query_dataframe tool with your pandas code as the argument.
    """)
    
    return query_prompt

In [13]:
def run_agent(question: str):
    """
    Run the complete query agent pipeline.
    
    Args:
        question: Natural language question about the DataFrame
        
    Returns:
        Dictionary containing question, code, and result.
    """
    logging.info(f"Processing question: {question}")
    
    try:
        model = init_model()
        query_prompt = create_prompt_templates()
        
        # Step 2: Format query prompt with schema information
        query_input = {
            "question": question,
            "columns": schema_info["columns"],
            "dtypes": schema_info["dtypes"],
            "sample_data": schema_info["sample_data"]
        }
        
        logging.info("Generating pandas code from question...")
        query_message = query_prompt.format_messages(**query_input)
        response = model.invoke(query_message)
        
        # Step 3: Extract pandas code from tool calls
        pandas_code = None
        if hasattr(response, 'tool_calls') and response.tool_calls:
            tool_call = response.tool_calls[0]
            pandas_code = tool_call["args"]["code"]
            
            
            result = query_dataframe(pandas_code)
        
            # Return the complete result
            return {
                "question": question,
                "pandas_code": pandas_code,
                "result": result,
            }
        else:
            logging.warning("No tool call made in response")
            return {
                "question": question,
                "error": "Failed to generate executable pandas code",
                "message": response.content
            }
    except Exception as e:
        logging.error(f"Error in agent execution: {e}")
        return {"question": question, "error": str(e)}

In [14]:
def main():
    """Main CLI entry point."""

    print("DataFrame Query Agent")

    schema_info = extract_schema_info(df)

    print(f"Loaded DataFrame with {schema_info['summary']['rows']} rows and {schema_info['summary']['columns']} columns")

    print("Type 'exit' to quit\n")

    while True:
        question = input("\n Ask questions about your data: ")

        if question.lower() in ('exit', 'quit'):
            print("Goodbye!")
            break

        result = run_agent(question)

        if "error" in result:
            print(f"\n Error: {result['error']}")
            if "message" in result:
                print(f"Additional info: {result['message']}")
        else:
            print(f"\n Generated pandas code:")
            print(f"```python\n{result['pandas_code']}\n```")

            print(f"\n Result:")
            print(result['result'])


In [15]:
if __name__ == "__main__":
    main()


DataFrame Query Agent
Loaded DataFrame with 1000 rows and 17 columns
Type 'exit' to quit



INFO - Processing question: which branch has maximum cash payment and which product line is in demand in each city?
INFO - Generating pandas code from question...
INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
C:\Users\nethi\AppData\Local\Temp\ipykernel_32456\3654945997.py:36: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = query_dataframe(pandas_code)



 Generated pandas code:
```python
(df[df['Payment'] == 'Cash'].groupby('Branch')['Total'].sum().idxmax(), df.groupby('City').apply(lambda x: x.groupby('Product line')['Quantity'].sum().idxmax()))
```

 Result:
('C', City
Mandalay      Sports and travel
Naypyitaw    Food and beverages
Yangon       Home and lifestyle
dtype: object)
Goodbye!
